In [1]:
import pandas as pd
from pathlib import Path

In [2]:
from fastai.tabular import *
from fastai.callbacks import ReduceLROnPlateauCallback, EarlyStoppingCallback, SaveModelCallback
from sklearn.metrics import roc_auc_score
import gc


ModuleNotFoundError: No module named 'fastai.callbacks'

In [3]:
dense161 = pd.read_csv("../input/cancer-densenet161-v2-for-ensemble/validation_0.976066529750824.csv")
dense161_test = pd.read_csv("../input/cancer-densenet161-v2-for-ensemble/submission_0.976066529750824.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../input/cancer-densenet161-v2-for-ensemble/validation_0.976066529750824.csv'

In [4]:
dense201 = pd.read_csv("../input/cancer-densenet201-v2-for-ensemble/validation_0.9749373197555542.csv")
dense201_test = pd.read_csv("../input/cancer-densenet201-v2-for-ensemble/submission_0.9749373197555542.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../input/cancer-densenet201-v2-for-ensemble/validation_0.9749373197555542.csv'

In [5]:
res50 = pd.read_csv("../input/cancer-resnet50-v2-for-ensemble/validation_0.9727705717086792.csv")
res50_test = pd.read_csv("../input/cancer-resnet50-v2-for-ensemble/submission_0.9727705717086792.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../input/cancer-resnet50-v2-for-ensemble/validation_0.9727705717086792.csv'

In [6]:
#trydf = pd.DataFrame({'dense161':dense161.ground_truth_label, 
#                      'dense201':dense201.ground_truth_label, 
#                      'res50':res50.ground_truth_label})
#trydf['1and2'] = trydf.dense161==trydf.dense201
#trydf['2and3'] = trydf.res50==trydf.dense201
#trydf['1and2'].nunique() == 1
#trydf['2and3'].nunique() == 1


In [7]:
def softmax_df(df, model_name, test=False):
    if test:
            df[model_name+'_0'] = np.exp(df['pred_0'])
            df[model_name+'_1'] = np.exp(df['pred_1'])
    else:
        df[model_name+'_0'] = np.exp(df['val_0'])
        df[model_name+'_1'] = np.exp(df['val_1'])
    df[model_name+'sum'] = df[model_name+'_0'] + df[model_name+'_1']
    df[model_name+'softmax'] = df[model_name+'_1'] / df[model_name+'sum']
    return df[model_name+'softmax']


In [8]:
#dense161_sm = softmax_df(dense161, 'dense161')
#dense201_sm = softmax_df(dense201, 'dense201')
#res50_sm = softmax_df(res50, 'res50')
#dense161_sm_test = softmax_df(dense161_test, 'dense161_test', True)
#dense201_sm_test = softmax_df(dense201_test, 'dense201_test', True)
#res50_sm_test = softmax_df(res50_test, 'res50_test', True)
#train = pd.DataFrame({'dense161_sm':dense161_sm, "dense201_sm":dense201_sm, "res50_sm":res50_sm, "y":dense161.ground_truth_label})
#test = pd.DataFrame({'dense161_sm':dense161_sm_test, "dense201_sm":dense201_sm_test, "res50_sm":res50_sm_test})
#test.y=0


In [9]:
dense161.head()


NameError: name 'dense161' is not defined

In [10]:
train = pd.DataFrame({'dense161_0':dense161.val_0, 'dense161_1':dense161.val_1, 
                      'dense201_0':dense201.val_0, 'dense201_1':dense201.val_1,
                      'res50_0':res50.val_0, 'res50_1':res50.val_1,
                      "y":dense161.ground_truth_label})
test = pd.DataFrame({'dense161_0':dense161_test.pred_0, 'dense161_1':dense161_test.pred_1, 
                      'dense201_0':dense201_test.pred_0, 'dense201_1':dense201_test.pred_1,
                      'res50_0':res50_test.pred_0, 'res50_1':res50_test.pred_1})
test.y=0


NameError: name 'dense161' is not defined

In [11]:
dep_var = 'y'
#cont_names = ['dense161_sm','dense201_sm', 'res50_sm']
cont_names = ['dense161_0', 'dense161_1', 'dense201_0', 'dense201_1', 'res50_0','res50_1']

data = (TabularList.from_df(train, cont_names=cont_names)
            .split_by_rand_pct(seed=47)
            .label_from_df(cols=dep_var)
            .add_test(TabularList.from_df(test, cont_names=cont_names))
            .databunch())


NameError: name 'TabularList' is not defined

In [12]:
def roc_score(inp, target):
    _, indices = inp.max(1)
    return torch.Tensor([roc_auc_score(target, indices)])[0]


In [13]:
learn = tabular_learner(data, layers=[10, 10, 10], metrics=[accuracy, roc_score],  ps=0.5, wd=1e-1, model_dir='./').to_fp16()


NameError: name 'tabular_learner' is not defined

In [14]:
#learn.lr_find()


In [15]:
#learn.recorder.plot()


In [16]:
from fastai.callbacks import ReduceLROnPlateauCallback, EarlyStoppingCallback, SaveModelCallback
ES = EarlyStoppingCallback(learn, monitor='roc_score',patience = 5)
RLR = ReduceLROnPlateauCallback(learn, monitor='roc_score',patience = 2)
SAVEML = SaveModelCallback(learn, every='improvement', monitor='roc_score', name='best')
learn.fit_one_cycle(20, 1e-3, callbacks = [ES, RLR, SAVEML])


ModuleNotFoundError: No module named 'fastai.callbacks'

In [17]:
learn.load('best')


NameError: name 'learn' is not defined

In [18]:
preds, _ = learn.get_preds(DatasetType.Test)
preds = torch.softmax(preds, dim=1)[:, 1].numpy()


NameError: name 'learn' is not defined

In [19]:
auc_val = learn.validate()[2].item()
auc_val


NameError: name 'learn' is not defined

In [20]:
sub = pd.read_csv("../input/histopathologic-cancer-detection/sample_submission.csv")
sub.head()


,id,label
0,acfe80838488fae3c89bd21ade75be5c34e66be7,0
1,a1991e73a9b676faddd2bd47c39754b14d1eb923,0
2,94fa32b29cc1c00403176c0795fffa3cfaa0f20e,0
3,0b820b71670c039dd0a51333d1c919f471a9e940,0
4,4b7a73f1fe1dafe2ffb7d2c0b83107f060b8d693,0


In [21]:
sub['label'] = preds


NameError: name 'preds' is not defined

In [22]:
sub.to_csv(f'submission_{auc_val}.csv', header=True, index=False)


NameError: name 'auc_val' is not defined

In [23]:
sub.head()


,id,label
0,acfe80838488fae3c89bd21ade75be5c34e66be7,0
1,a1991e73a9b676faddd2bd47c39754b14d1eb923,0
2,94fa32b29cc1c00403176c0795fffa3cfaa0f20e,0
3,0b820b71670c039dd0a51333d1c919f471a9e940,0
4,4b7a73f1fe1dafe2ffb7d2c0b83107f060b8d693,0
